In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression

In [103]:
n_actions = 5
n_features = 10
n_data_list = [125, 250, 500, 1000, 2000, 4000, 8000, 16000, 32000]

In [104]:
Wx0 = np.random.normal(size=(n_actions, n_features))
Bx0 = np.random.normal(size=(n_actions, 1))

Wx = np.random.normal(size=(n_actions, n_features))
Bx = np.random.normal(size=(n_actions, 1))

Wxa_x = np.random.normal(size=(n_features,))
Wxa_a = np.random.normal(size=(n_actions,))

In [113]:
class Estimator():
    def __init__(self, n_data):
        self.x = np.random.normal(size=(n_data, n_features))

    def sigmoid(self, x):
        return 1./(1.+np.exp(x))

    def feature2action0(self, x):
        prob_a = self.sigmoid(Wx0@x.T+Bx0).T
        prob_a = prob_a/prob_a.sum(axis=1, keepdims=True)
        return prob_a

    def feature2action(self, x):
        prob_a = self.sigmoid(Wx@x.T+Bx).T
        prob_a = prob_a/prob_a.sum(axis=1, keepdims=True)
        return prob_a

    def sample_actions(self, prob_a):
        data_size, action_size = prob_a.shape
        actions = np.array([np.random.choice(action_size, p=prob_a[i]) for i in range(data_size)])
        result = np.zeros_like(prob_a)
        result[np.arange(data_size), actions] = 1
        return result, actions

    def action2reward(self, x, a):
        return (Wxa_x@x.T+Wxa_a@a.T).T

    def data_generator(self):
        prob_a0 = self.feature2action0(self.x)
        a0, actions0 = self.sample_actions(prob_a0)
        r0 = self.action2reward(self.x, a0)

        prob_a = self.feature2action(self.x)
        a, actions = self.sample_actions(prob_a)
        r = self.action2reward(self.x, a)

        model = LogisticRegression(multi_class='multinomial', max_iter=200)
        model.fit(X=self.x, y=actions0)

        a_pred0 = model.predict_proba(self.x)
    
        self.prob_a = prob_a
        self.actions0 = actions0

        self.a_pred0 = a_pred0
        self.r0 = r0
    
    def DR_estimator(self):
        self.data_generator()
        
        r0 = self.r0
        a_pred0 = self.a_pred0

        model = LinearRegression()
        model.fit(X=np.concatenate([self.x, a_pred0], axis=1), y=r0)

        r_pred0 = model.predict(X=np.concatenate([self.x, a_pred0], axis=1))

        return np.sqrt(((r0-r_pred0)**2).mean())

    def IPS_estimator(self):
        self.data_generator()
        
        r0 = self.r0
        a_pred0 = self.a_pred0
        prob_a = self.prob_a
        actions0 = self.actions0

        r_pred0 = r0 *  prob_a[actions0] / a_pred0[actions0]

        return np.sqrt(((r0-r_pred0)**2).mean())

In [114]:
for n_data in n_data_list:
    estimator = Estimator(n_data)
    print(n_data, estimator.DR_estimator())
    # print(n_data, estimator.IPS_estimator())

125 0.3056273807602827


ValueError: operands could not be broadcast together with shapes (125,) (125,5) 

42.66458776552922